Our data comes from this paper:
https://ai.googleblog.com/2022/03/using-deep-learning-to-annotate-protein.html

# Initializing variables

In [1]:
dataset_folder = "../google_prot_fns/"
filters = 64
epochs = 60
batch_size = 128
protein_len = 200
protein_family_number = 600

# Import Packages

In [3]:
# import tensorflow as tf
from sklearn import preprocessing
# from keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from keras.layers import add, maximum, minimum
# import tensorflow_probability as tfp
# tfd = tfp.distributions 
# tfpl = tfp.layers
# from keras import regularizers
import pandas as pd
import numpy as np
import os

# Read Dataset

In [4]:
def read_data(name_sub_folder):
  full_data = []
  for f in os.listdir(os.path.join(dataset_folder, name_sub_folder)):
    data = pd.read_csv(os.path.join(dataset_folder, name_sub_folder, f))
    full_data.append(data)
  return pd.concat(full_data)  

In [5]:
dataset_train = read_data("train")
dataset_train.head()

,family_id,sequence_name,family_accession,aligned_sequence,sequence
0,HTH_12,D7DT16_METV3/7-72,PF08461.10,IAILNLLSKYDKPIGAKIIAEDLKLR.GYDIGERAVRYHLQSMDDD...,IAILNLLSKYDKPIGAKIIAEDLKLRGYDIGERAVRYHLQSMDDDE...
1,NFACT-R_1,D6GU87_PARA5/390-500,PF05670.13,KFRFFS...............TSLN......KLCIIGKDVNQ.NES...,KFRFFSTSLNKLCIIGKDVNQNESLIQKHAEKGDIVGHADVFGSPF...
2,Nop53,B2ASQ0_PODAN/21-405,PF07767.11,SRKGKKAWRKNVDVT...DITKGLDQLN.TQKIIGG.VIAEKDSAD...,SRKGKKAWRKNVDVTDITKGLDQLNTQKIIGGVIAEKDSADLFVLD...
3,DUF72,D3SM48_THEAH/20-240,PF01904.18,YPPRLRR.DE.WIKYYAKH..FVVLELNATF........YG.FPDR...,YPPRLRRDEWIKYYAKHFVVLELNATFYGFPDRGSIKSMLSRVKDL...
4,Glyco_hydro2_C5,U2B330_9CLOT/643-745,PF18565.1,LRLTTDKEQLSADGKSLAFLTITAVDREGRTVENAVNRILVKVEGA...,LRLTTDKEQLSADGKSLAFLTITAVDREGRTVENAVNRILVKVEGA...


In [6]:
dataset_test = read_data("test")
dataset_test.head()

,family_id,sequence_name,family_accession,aligned_sequence,sequence
0,HD,Q9X1N3_THEMA/52-163,PF01966.22,RFAHSLGTMHVAGLYARNLFK......................ESD...,RFAHSLGTMHVAGLYARNLFKESDRIRIVRLAALLHDVGHGPFSHQ...
1,ChW,K4QXH7_STRDJ/317-351,PF07538.11,HLSGSG........W.........Q.......K.....P..V.CDG...,HLSGSGWQKPVCDGTLAGTTGQEKPIKAINIAVTG
2,7TM_GPCR_Srd,A8WZJ1_CAEBR/10-305,PF10317.9,LSIFYPIFFVITLIAQLFLMFLIIKHSPKSIHMLRIILGLTCIFQI...,LSIFYPIFFVITLIAQLFLMFLIIKHSPKSIHMLRIILGLTCIFQI...
3,Integrin_beta,W5MQ40_LEPOC/123-370,PF00362.18,TEDYPVDVYYLMDLSASMSDDL.....EMIKDLGSTLSRE..MAKL...,TEDYPVDVYYLMDLSASMSDDLEMIKDLGSTLSREMAKLTSKFRLG...
4,Rimk_N,A0A0U2RI95_9ALTE/1-94,PF18030.1,MRIAVLSRNANL..YSTRRLKEEGEAHGHEMHIIDPLKCYMNINMQ...,MRIAVLSRNANLYSTRRLKEEGEAHGHEMHIIDPLKCYMNINMQSS...


In [7]:
dataset_dev = read_data("dev")
dataset_dev.head()

,family_id,sequence_name,family_accession,aligned_sequence,sequence
0,ASL_C,K4A843_SETIT/406-520,PF08328.11,SRMQRDLTDSTVLRNLGVGLGHSLLAYKATMRGISKVQVNESRLAE...,SRMQRDLTDSTVLRNLGVGLGHSLLAYKATMRGISKVQVNESRLAE...
1,DUF1735,E7RLQ4_9BACT/29-153,PF08522.10,YKH...V............FSFI.............CN.TD........,YKHVFSFICNTDHVSQKMFNLSDTLRTGYISLSMGGSNPTDEDVTV...
2,DUF4846,R5X9T4_9CLOT/55-294,PF16138.5,IKERYDVPEGYERVSTEENSFGEFLRNSKLEKYGEKVKYYDGRT.....,IKERYDVPEGYERVSTEENSFGEFLRNSKLEKYGEKVKYYDGRTKD...
3,Candida_ALS,A5DJ08_PICGU/1856-1887,PF05792.13,TTLT.STWN.....GT...V..TTT....TT.FS...ASV.........,TTLTSTWNGTVTTTTTFSASVGGTETVIVEIP
4,Seipin,B6H3Z9_PENRW/52-254,PF06775.14,TFLIGISAIAYGVF.YFNFIP.TVG.LEREVHLQF..G...........,TFLIGISAIAYGVFYFNFIPTVGLEREVHLQFGDGNPWGTAHFDSE...


In [ ]:
# dataset_train = pd.concat([dataset_train, dataset_test])

In [ ]:
# dataset_train.head()

In [8]:
dataset_train['family_accession'].value_counts()

PF13649.6     3637
PF00560.33    1927
PF13508.7     1761
PF06580.13    1537
PF02397.16    1528
              ... 
PF17055.5        1
PF09096.10       1
PF10894.8        1
PF03507.13       1
PF17606.2        1
Name: family_accession, Length: 17929, dtype: int64

## Select until 600 families

In [9]:
classes = dataset_train['family_accession'].value_counts()[:protein_family_number].index

In [10]:
def return_from_dataset(dataset, classes):
  return [dataset.loc[dataset['family_accession'].isin(classes)].reset_index(), classes]

In [11]:
[dataset_train, classes] = return_from_dataset(dataset_train, classes)

In [12]:
dataset_train.head()

,index,family_id,sequence_name,family_accession,aligned_sequence,sequence
0,3,DUF72,D3SM48_THEAH/20-240,PF01904.18,YPPRLRR.DE.WIKYYAKH..FVVLELNATF........YG.FPDR...,YPPRLRRDEWIKYYAKHFVVLELNATFYGFPDRGSIKSMLSRVKDL...
1,9,Glyco_tran_WecB,F4LWT6_TEPAE/64-231,PF03808.13,ADI.CLPDGIGVVIASK.I...L..G.K.PL......VERTTGF.....,ADICLPDGIGVVIASKILGKPLVERTTGFDFMMKVLGMADKESLRL...
2,13,CPSase_sm_chain,B9KXM6_THERP/10-136,PF00988.22,..ALVLEDG.RI.FPGT.PFG...AR..............RPA......,ALVLEDGRIFPGTPFGARRPAEGEAVFTTVMTGYQEVVTDPSFYGQ...
3,16,ZT_dimer,R5V116_9BACT/209-284,PF16916.5,..SE.KIVLQVKEIV.R.Q....EM.G.VVAYPHHFHIH.HYG..D...,SEKIVLQVKEIVRQEMGVVAYPHHFHIHHYGDYAEFIFHIKVPGEK...
4,21,MurB_C,R5EF29_9FIRM/204-302,PF02873.16,EIWAKMKELIARR.TASQ........P.L.....GV......PSAG...,EIWAKMKELIARRTASQPLGVPSAGSAFKRPTGGYAAALIEEAGLK...


In [13]:
dataset_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329768 entries, 0 to 329767
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   index             329768 non-null  int64 
 1   family_id         329768 non-null  object
 2   sequence_name     329768 non-null  object
 3   family_accession  329768 non-null  object
 4   aligned_sequence  329768 non-null  object
 5   sequence          329768 non-null  object
dtypes: int64(1), object(5)
memory usage: 15.1+ MB


In [14]:
classes

Index(['PF13649.6', 'PF00560.33', 'PF13508.7', 'PF06580.13', 'PF02397.16',
       'PF00677.17', 'PF01035.20', 'PF02417.15', 'PF13472.6', 'PF00684.19',
       ...
       'PF02570.15', 'PF00813.20', 'PF00681.20', 'PF00447.17', 'PF01176.19',
       'PF00338.22', 'PF07963.12', 'PF12392.8', 'PF02595.15', 'PF00858.24'],
      dtype='object', length=600)

In [15]:
[dataset_dev, classes_dev] = return_from_dataset(dataset_dev, classes) 

In [16]:
dataset_dev.head()

,index,family_id,sequence_name,family_accession,aligned_sequence,sequence
0,3,Candida_ALS,A5DJ08_PICGU/1856-1887,PF05792.13,TTLT.STWN.....GT...V..TTT....TT.FS...ASV.........,TTLTSTWNGTVTTTTTFSASVGGTETVIVEIP
1,5,zf-HYPF,F8B255_FRADG/188-221,PF07503.12,MCSACAA.EYHDPAD...RRFHAQPVCCPDCGPRLSFV,MCSACAAEYHDPADRRFHAQPVCCPDCGPRLSFV
2,6,Hexokinase_1,G8ZUD9_TORDC/26-221,PF00349.21,QIHGLEVIFGTTPEKL.RAVVKHFI.GEL....................,QIHGLEVIFGTTPEKLRAVVKHFIGELNKGLSKKGGNIPMIPGWVM...
3,12,ATP-synt_A,F2LVN4_HIPMA/20-215,PF00119.20,LVFSWFMILVII....SLA.....LI.......VRSSLK.LM.......,LVFSWFMILVIISLALIVRSSLKLMPEGIQNVAESVVYGIYSFVED...
4,13,ACBP,Q382P9_TRYB2/692-770,PF00887.19,.TAFAAAQQAF..A........RI.........VGM..V..........,TAFAAAQQAFARIVGMVSNDMKLNIYALYKQATVGDINISRPWLTD...


In [17]:
def features_target_split(dataset):
  return [dataset['sequence'], dataset['family_accession']]

In [18]:
[features, target] = features_target_split(dataset_train)
[features_dev, target_dev] = features_target_split(dataset_dev)

In [19]:
len(classes)

600

## Preprocessing dataset

In [ ]:
def cut_protein_sequence(sequence):
  return sequence.apply(lambda seq: [seq[i:i+ 1] for i in range(0, len(seq), 1)])

In [ ]:
features = cut_protein_sequence(features)
features_dev = cut_protein_sequence(features_dev)

In [ ]:
features

In [ ]:
features_dev

In [ ]:
label_ = preprocessing.LabelEncoder()
label_.fit(target)

In [ ]:
label_.classes_

In [ ]:
target = label_.transform(target)

In [ ]:
target_dev = label_.transform(target_dev)

In [ ]:
target = tf.keras.utils.to_categorical(target)
target_dev = tf.keras.utils.to_categorical(target_dev)

In [ ]:
tokenizer_seq=  Tokenizer(num_words = 2000)
tokenizer_seq.fit_on_texts(features)

In [ ]:
features = tokenizer_seq.texts_to_sequences(features)
features_dev = tokenizer_seq.texts_to_sequences(features_dev)

In [ ]:
features = pad_sequences(features, maxlen = 200, padding = "post", truncating = "post")
features.shape

In [ ]:
features_dev = pad_sequences(features_dev, maxlen = features.shape[1], padding = "post", truncating = "post")

In [ ]:
features_dev.shape

In [ ]:
print(tokenizer_seq.word_index)

In [ ]:
vocabulary = len(tokenizer_seq.word_index)

# CNN Model

In [ ]:
def resnet_con(inputs, filters, activation = "relu", strides = 1):
  x = tf.keras.layers.BatchNormalization()(inputs)
  x = tf.keras.layers.Conv1D(filters, 3, strides = strides, activation = activation, padding = "same",
                             kernel_regularizer=tf.keras.regularizers.L2(0.01))(x)
  x = tf.keras.layers.Activation("relu")(x)
  return x

In [ ]:
filters = 64
input = tf.keras.layers.Input(shape=(features.shape[1], ))
x = tf.keras.layers.Embedding(vocabulary + 1, 150)(input)
x1 = tf.keras.layers.AveragePooling1D()(x)
x2 = tf.keras.layers.MaxPooling1D()(x)
x = tf.keras.layers.concatenate([x1, x2])
x = tf.keras.layers.Dropout(0.3)(x)
x = resnet_con(x, filters)
for stack in range(2):
  for block in range(3):
    strides = 1
    if stack > 0 and block == 0:
      strides = 2
    y = resnet_con(x, filters, activation = None, strides = strides)
    y = tf.keras.layers.BatchNormalization()(y)
    y = tf.keras.layers.Conv1D(filters, 3, activation = None, padding = "same",
                               kernel_regularizer=tf.keras.regularizers.L2(0.01))(y)                       
    if stack > 0 and block == 0:
      strides = 2
      x = tf.keras.layers.Conv1D(filters , 1, activation = None, strides = strides, padding = "same",
                                 kernel_regularizer=tf.keras.regularizers.L2(0.01))(x)
    x = add([x, y])
    x = tf.keras.layers.Activation("relu")(x)
  filters *=2
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.7)(x)
x = tf.keras.layers.Dense(1024, activation = "relu")(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(512, activation = "relu")(x)
x = tf.keras.layers.Dropout(0.45)(x)
y = tf.keras.layers.Dense(len(classes), activation ="softmax",
                          kernel_regularizer=tf.keras.regularizers.L2(0.0001))(x)
m = tf.keras.models.Model(input, y)
m.compile(optimizer = tf.optimizers.Adam(learning_rate=0.001), loss = "categorical_crossentropy", metrics = ['accuracy'])
m.summary()     

In [ ]:
tf.keras.utils.plot_model(m, show_shapes = True, show_layer_activations= True)

# Training Model

In [ ]:
history = m.fit(features, target, epochs = 60, batch_size = 128,
                validation_data = (features_dev, target_dev), 
               callbacks = [tf.keras.callbacks.EarlyStopping(patience = 6, 
                                                              monitor = 'val_loss', 
                                                              mode = 'min', 
                                                              restore_best_weights=True)])

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (12, 5))
plt.plot(history.history['loss'], label = "loss")
plt.plot(history.history['val_loss'], label = "validation loss")
plt.legend()

In [ ]:
plt.figure(figsize = (12, 5))
plt.plot(history.history['accuracy'], label = "accuracy")
plt.plot(history.history['val_accuracy'], label = "validation accuracy")
plt.legend()

# Result

In [ ]:
m.evaluate(features_dev, target_dev)

In [ ]:
y_pred = m.predict(features_dev)

In [ ]:
y_pred = np.argmax(y_pred, axis = 1)

In [ ]:
print(tf.math.confusion_matrix(
    np.argmax(target_dev, axis = 1),
    y_pred,
))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
m.save("drive/MyDrive/pfam.h5")
m.save_weights("drive/MyDrive/pfam_weights.h5")

In [ ]:
import pickle
with open('drive/MyDrive/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer_seq, handle, protocol=pickle.HIGHEST_PROTOCOL)